In [17]:
#bibliotecas necesarias para el funcionamiento
import pandas                  as pd                     #manejo y manipulación de dataframes
import scipy
import numpy                   as np                     #soporte matricial y vectorial
import matplotlib.pyplot       as plt                    #graficos
import joblib
from   sklearn.model_selection import GridSearchCV       #optimización de hiperparametros
from   sklearn                 import metrics            #metricas de validación
from   sklearn.model_selection import train_test_split   #segmentación de muestra
from   sklearn.linear_model    import LogisticRegression #clasificador 
from   sklearn.impute          import SimpleImputer      
from   sklearn.compose         import ColumnTransformer
from   sklearn.pipeline        import Pipeline
from   sklearn.preprocessing   import LabelEncoder
from   sklearn.preprocessing   import OneHotEncoder,MinMaxScaler
from   sklearn.preprocessing   import KBinsDiscretizer
from   sklearn.metrics         import accuracy_score, precision_score, recall_score, f1_score

In [11]:
df=pd.read_csv("incumplimiento.csv",sep=",",header=0) #hacer la lectura de datos
df.head(3)

,Folio,Incumplimiento,Estudiante,Saldo,Ingreso
0,1,No,No,729.526495,44361.625074
1,2,No,Yes,817.180407,12106.134700
2,3,No,No,1073.549164,31767.138947


In [12]:
y=LabelEncoder().fit_transform(df["Incumplimiento"])
X=df.drop(["Incumplimiento","Folio"],axis=1)
X.head(1)

,Estudiante,Saldo,Ingreso
0,No,729.526495,44361.625074


In [14]:
#crea un pipeline para generar el modelo
def modelo(X_train, y_train,numeric_features,categorical_features):
    numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
     ('scaler', MinMaxScaler())    
    ])
    categorical_transformer = Pipeline(steps=[
        #imputacion por el valor más frecuente
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    #onehot encoder para las varibales categoricas    
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    #el pipeline con el procesamienot y el claificador
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(fit_intercept=True,solver='lbfgs',tol=1e-5,max_iter=75))])
    modelo=clf.fit(X_train, y_train)
    return modelo

In [18]:
#El score obtenido con el modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
categorical_features = ["Estudiante"]
numeric_features=['Saldo','Ingreso']
modelo=modelo(X_train, y_train,numeric_features,categorical_features)
print("model score: %.3f" % modelo.score(X_test, y_test))


model score: 0.968


In [20]:
#la matriz de confusion
cm = metrics.confusion_matrix(y_test, y_pred)
print(cm) 

[[2885    2]
 [  93   20]]


In [19]:
y_pred=modelo.predict(X_test)
precision_score(y_test, y_pred, average='binary')

0.9090909090909091